<a href="https://colab.research.google.com/github/cbachen1997/CAUstudy/blob/master/GetTriplet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#挂载谷歌云盘
from google.colab import drive
drive.mount('/content/gdrive')
####google云盘授权#####
##每个notebook执行一次###
__author__='CBA'
from google.colab import drive

#增加PyDrive操作库
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#授权登录
auth.authenticate_user()
gauth=GoogleAuth()
gauth.credentials=GoogleCredentials.get_application_default()
drive=GoogleDrive(gauth)

In [ ]:
import os
os.chdir("/content/gdrive/MyDrive/CM")
print (os.path.abspath('.')) #获取当前工作目录路径
print(os.listdir('.'))

/content/gdrive/MyDrive/CM
['models.py', 'data_util.py', 'main.py', 'ops.py', 'img', 'roi_patches', 'model.ipynb', 'unlabeled_patches', 'ROI2Patches.ipynb', 'GetTriplet.ipynb']


In [ ]:
from __future__ import print_function, division
import tensorflow as tf

from sklearn.utils import shuffle
import numpy as np
import random
from tqdm import *
import time
import tensorflow.keras as K
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.python.keras.layers.advanced_activations import LeakyReLU


In [ ]:
X_train_path = '/content/gdrive/MyDrive/CM/roi_patches/test_X_new.npy'
Y_train_path = '/content/gdrive/MyDrive/CM/roi_patches/test_Y_new.npy'

In [ ]:
X_train = np.load(X_train_path)
Y_train = np.load(Y_train_path)
#454个点，231个正例，223个负例

In [ ]:
X_pos=[]
Y_pos=[]
X_neg=[]
Y_neg=[]

In [ ]:
for i in range(len(Y_train)):
  if (Y_train[i]==1):
    X_pos.append(X_train[i])
    Y_pos.append(Y_train[i])
  else:
    X_neg.append(X_train[i])
    Y_neg.append(Y_train[i])
X_pos = np.asarray(X_pos, dtype=np.float32)
Y_pos = np.asarray(Y_pos, dtype=np.float32)
Y_pos = np.expand_dims(Y_pos,axis=1)
X_neg = np.asarray(X_neg, dtype=np.float32)
Y_neg = np.asarray(Y_neg, dtype=np.float32)
Y_neg = np.expand_dims(Y_neg,axis=1)

In [ ]:
#简单搭个网络方便跑模型
inputshape=(11,11,7)
num=25#正负各抽25个
alpha=0.05

In [ ]:
def SimpleNet(input_shape):
  """
  实现一个用于三元组的简单模型
  
  参数：
      input_shape - 输入的数据的维度
  返回：
      model - 创建的Keras的模型
      
  """

  X_input=Input(input_shape)
  print(X_input.shape)
  X = ZeroPadding2D((3, 3))(X_input)
  X = Conv2D(32, (7, 7), strides=(1, 1), name='conv0',kernel_initializer='he_normal')(X)
  X = BatchNormalization(axis=3, name='bn0')(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2, 2), name='max_pool')(X)
  X = Flatten()(X)#向量化
  #最后输出128维向量便于计算度量
  X = Dense(128,name='dense_layer')(X)
  # L2 normalization
  # X = Lambda(lambda  x: K.l2_normalize(x,axis=1))(X)
  model = Model(inputs=X_input,outputs=X,name='SimpleModel')

 
  return model

In [ ]:
#三元组部分
#随机抽取patch
def sampling(datanpy,num=num):
  #encode list
  list_for_sample=[ i for i in range(datanpy.shape[0])]
  samples=random.sample(list_for_sample,num)
  samples_npy=np.zeros((num,datanpy.shape[1],datanpy.shape[2],datanpy.shape[3]))
  count=0
  for i in samples:
    samples_npy[count,:,:,:]=datanpy[i,:,:,:]
    count+=1
  return samples_npy
#编码
def embedding(datanpy,model):
  length  = datanpy.shape[0]
  embed_array = np.zeros((datanpy.shape[0],128))
  for i in range(length):
    embedding=model.predict_on_batch(datanpy)
  return embedding
#度量计算
# from scipy.spatial.distance import pdist
# from scipy.spatial.distance import squareform
#距离
# def D_mat(X):
#   """
#   X: batch中的所有embedding
#   """
#   distA=pdist(X,metric='euclidean') 
#   distB = squareform(distA)
#   return distB
#获取hard_triplet
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import pdist
def get_hard_triplet(X_p,X_n,model):
  """
  X_p:原始正例
  X_n:原始负例
  """
  num_=X_p.shape[0]
  pos_embed=embedding(X_p,model)
  neg_embed=embedding(X_n,model)
  hard_triplet=np.zeros((num_*2,3,X_p.shape[1],X_p.shape[2],X_p.shape[3]))
  count=0
  print('构建正例样本困难三元组')
  for i in range(num_):
    anchor=pos_embed[i,:].reshape((1,-1))
    anchor = np.repeat(anchor, num_, axis=0)
    #寻找困难正样本,距离最大正
    temp_dis=np.sum((anchor-pos_embed)*(anchor-pos_embed),axis=1)
    loc_p=temp_dis.argmax()
    hard_pos=X_p[loc_p,:,:,:]
    #寻找困难负样本，距离最小负
    temp_dis=np.sum((anchor-neg_embed)*(anchor-neg_embed),axis=1)
    loc_n=temp_dis.argmin()
    hard_neg=X_n[loc_n,:,:,:]
    count+=1
    hard_triplet[i,0,:,:,:]=X_p[i,:,:,:]
    hard_triplet[i,1,:,:,:]=hard_pos
    hard_triplet[i,2,:,:,:]=hard_neg

  print('构建负例样本困难三元组')
  for i in range(num_):
    anchor=neg_embed[i,:].reshape((1,-1))
    anchor = np.repeat(anchor, num_, axis=0)
    #寻找正样本,距离最大负
    temp_dis=np.sum((anchor-neg_embed)*(anchor-neg_embed),axis=1)
    loc_p=temp_dis.argmax()
    hard_pos=X_n[loc_p,:,:,:]
    #寻找负样本，距离最小正
    temp_dis=np.sum((anchor-pos_embed)*(anchor-pos_embed),axis=1)
    loc_n=temp_dis.argmin()
    hard_neg=X_p[loc_n,:,:,:]
    hard_triplet[i,0,:,:,:]=X_p[i,:,:,:]
    hard_triplet[i,1,:,:,:]=hard_pos
    hard_triplet[i,2,:,:,:]=hard_neg
  return hard_triplet
  

In [ ]:
#构建困难三元组
def process_triplet(pos_sample,neg_sample,model,num=num):
  #抽样
  X_p=sampling(pos_sample,num=num)
  X_n=sampling(neg_sample,num=num)
  #获取困难三元组
  hard_tri=get_hard_triplet(X_p,X_n,model)
  return hard_tri

In [ ]:
#三元组损失计算
def triplet_loss(triplet,alpha=alpha):
  """
  triplet=三元组
  """
  anchor,positive,negative=triplet[0],triplet[1],triplet[2]
  #第一步：计算"anchor" 与 "positive"之间编码的距离，这里需要使用axis=-1
  pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)),axis=-1)#二范数，-1代表倒数第一个维度即128
  #第二步：计算"anchor" 与 "negative"之间编码的距离，这里需要使用axis=-1
  neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)),axis=-1)
  #第三步：减去之前的两个距离，然后加上alpha
  basic_loss = tf.add(tf.subtract(pos_dist,neg_dist),alpha)
  #通过取带零的最大值和对训练样本的求和来计算整个公式
  loss = tf.reduce_sum(tf.maximum(basic_loss,0))

  return loss

In [ ]:
def create_train_model(model,alpha):
  y_pred=[]
  anchor_input = Input(inputshape) 
  positive_input = Input(inputshape)
  negative_input = Input(inputshape)
  anchor_out = Model(anchor_input, model(anchor_input))
  positive_out = Model(positive_input, model(positive_input))
  negative_out = Model(negative_input, model(negative_input))
  y_pred.append(anchor_out.output)
  y_pred.append(positive_out.output)
  y_pred.append(negative_out.output)

  loss = Lambda(lambda x: triplet_loss(x, alpha), output_shape=[1])(y_pred)
  # y_true=np.zeros((total,3)).tolist() 
  # alpha=0.2
  return Model(inputs = [anchor_input,positive_input,negative_input], outputs=loss)

In [ ]:
def train_model(model):
  tic = time.clock()
  #抽三元组次数
  num_times=1
  mm=create_train_model(model, 0.2)
  optm=K.optimizers.Adam(lr=1e-5)
  mm.compile(optimizer=optm,loss=lambda t,p:p)
  # model_checkpoint = ModelCheckpoint('/content/gdrive/MyDrive/CM', monitor=None, verbose=1, 
  #                                      save_best_only=True, save_weights_only=True,period=10)
  for i in tqdm(range(num_times)):
    hard_tri=process_triplet(X_pos,X_neg,model,num=num)
    y_label=np.zeros((len(hard_tri),1))#y用不着
    xx = [
          hard_tri[:,0,:,:,:],
          hard_tri[:,1,:,:,:],
          hard_tri[:,2,:,:,:]
    ]
    mm.fit(x=xx,y=y_label,batch_size=5,epochs=500)
  model.save('/content/gdrive/MyDrive/CM/temp.h5')

In [ ]:
Smodel=SimpleNet(inputshape)
mmodel=create_train_model(Smodel,alpha=alpha)

In [ ]:
train_model(Smodel)

In [ ]:
unlabeled_path='/content/gdrive/MyDrive/CM/unlabeled_patches/unlabeled_X.npy'
weight_='/content/gdrive/MyDrive/CM/temp.h5'
unlabeled_pool=np.load(unlabeled_path)
Smodel.load_weights(weight_)
positive_embed=embedding(X_pos,Smodel)
negative_embed=embedding(X_neg,Smodel)


In [ ]:
#半监督部分
#mixup获取伪标签
scale=200#每次抽取的无标签数量
topk=25
def mixup(p_embed,n_embed,unlabeled,model,scale=scale):
  Smodel.load_weights(weight_)
  #随机从无标签池中抽取样本
  temp_unlabeled=sampling(unlabeled,scale)
  #开始mixup
  count_all_label=[]
  for i in tqdm(range(len(temp_unlabeled))):
    data_enhanced=np.zeros((7,11,11,7))
    data_enhanced[0,]=unlabeled[i,]
    #水平垂直对角翻转
    data_enhanced[1,]=np.flip(unlabeled[i,],axis=0)
    data_enhanced[2,]=np.flip(unlabeled[i,],axis=1)
    data_enhanced[3,]=np.flip(unlabeled[i,],axis=2)
    #增加噪声(轻噪，重噪)
    noise = np.random.normal(0.0, 0.01, size=unlabeled[i,].shape)
    data_enhanced[4,]=i+noise
    noise = np.random.normal(0.0, 0.05, size=unlabeled[i,].shape)
    data_enhanced[5,]=i+noise
    #随机旋转
    k = np.random.randint(4)
    data_enhanced[6,]=np.rot90(unlabeled[i,],k=k)
    #增强结果的embed
    enhance_embed=embedding(data_enhanced,model)
    #计算度量获取伪标签
    pseudo_label=[]
    for j in range(len(enhance_embed)):
      #扩充向量维度，计算距离，找最小距离
      dis_list=[]
      #计算正样本池
      temp=np.repeat(enhance_embed[j,].reshape((1,-1)), p_embed.shape[0], axis=0)
      temp_dis_mat=np.sum((temp-p_embed)*(temp-p_embed),axis=1)
      temp_p_dis=np.min(temp_dis_mat) 
      #计算负样本池
      temp=np.repeat(enhance_embed[j,].reshape((1,-1)), n_embed.shape[0], axis=0)
      temp_dis_mat=np.sum((temp-n_embed)*(temp-n_embed),axis=1)
      temp_n_dis=np.min(temp_dis_mat)
      #伪样本标签获取
      dis_list.append(temp_n_dis)
      dis_list.append(temp_p_dis)
      label = dis_list.index(min(dis_list))      
      pseudo_label.append(label)
    counts = np.bincount(pseudo_label)
    plabel= np.argmax(counts)
    acc=counts[plabel]/len(pseudo_label)
    count_all_label.append((plabel,acc))

  #返回batch所有无标签样本和对应伪标签及概率
  return temp_unlabeled,count_all_label
#获取pseudo triplet
def ranking(unlabeled_sample,prob,topk=topk):
  """
  unlabeled_sample:batchsize的抽样无标签样本
  prob:对应伪标签和概率
  """
  #类别分开放
  p_pseudo_list=[]
  n_pseudo_list=[]
  pof200_list=[]
  nof200_list=[]
  for i in range(len(prob)):
    if prob[i][0]==1:
      p_pseudo_list.append(prob[i])
      pof200_list.append(unlabeled_sample[i,].tolist())
    else:
      n_pseudo_list.append(prob[i])
      nof200_list.append(unlabeled_sample[i,].tolist())
  p_pseudo_array=np.asarray(p_pseudo_list)
  n_pseudo_array=np.asarray(n_pseudo_list)
  pof200_array=np.asarray(pof200_list)
  nof200_array=np.asarray(nof200_list)
  #按置信度排序
  p_rank=np.argsort(p_pseudo_array[:,1],axis=0)
  n_rank=np.argsort(n_pseudo_array[:,1],axis=0)
  #分类样本
  last_pos=[]
  last_neg=[]
  for i in range(topk):
    last_pos.append(pof200_array[p_rank[i]].tolist())
    last_neg.append(nof200_array[n_rank[i]].tolist())
  last_pos=np.asarray(last_pos)
  last_neg=np.asarray(last_neg)
  #返回mixup获得的置信度前25的伪标签样本作为伪标签样本池
  return last_pos,last_neg

In [ ]:
unlabeled_batch,class_prob=mixup(positive_embed,negative_embed,unlabeled_pool,Smodel,scale=scale)

100%|██████████| 200/200 [00:07<00:00, 28.38it/s]


In [ ]:
pseudo_pos,pseudo_neg=ranking(unlabeled_batch,class_prob)

In [ ]:
pseudo_hard_tri=process_triplet(pseudo_pos,pseudo_neg,Smodel,num=10)#伪样本池出10*2个困难三元组，降低权重

构建正例样本困难三元组
构建负例样本困难三元组
